# The Quarterly Japanese Economic Model (Q-JEM)
This workbook implement the "The Quarterly Japanese Economic Model (Q-JEM): 2019 version". 

Press **Space** to proceed. Press **Shift-space** to go back 

In code cells you press Shift-Enter to evaluate your code.

You can always use smaller / larger fonts with keyboard shortcuts like **Alt +** and **Alt -** or similar (it could be Ctrl instead of Alt depending on the platform you are on).

If you want to leave the slideshow and return to the notebook, just press the upper left **X** 


At http://www.boj.or.jp/en/research/wps_rev/wps_2019/wp19e07.htm/ you will find the working paper describing 
the model and a zipfile containing all the relevant information needed to use the model. 

The model logic has been transformed from Eview equation to ModelFlow Business logic and the dataseries has been transformed to a Pandas Dataframe. 

In this workbook the impulse responses from the working paper section 3.1.1, 3.1.2, 3.1.3, and 3.1.4 has been recreated. 

The quarters has been rebased to 2001q1 to 2009q4

## Import Python libraries

In [ ]:
import pandas as pd 

import modelmf
from modelclass import model
model.modelflow_auto()

## Create model and dataframe 

In [ ]:
mqjem, baseline = model.modelload('qjem.pcim',run=1)

## Define some variable labels 
This gives more meaningful legends 

In [ ]:
legend = {
'GDP'   : 'Real gross domestic product, S.A.',
'CP'    : 'Real private consumption, S.A.',
'U'     : 'Unemployment rate, S.A.',
'PGDP'  : 'GDP deflator',
'USGDP' : 'Real gross domestic product of the United States, S.A.',
'NUSGDP': 'Output gap of the rest of the world',
'EX': 'Real exports of goods and services, S.A.',
'IM' : 'Real imports of goods and services, S.A.',
'INV' : 'Real private non-residential investment, S.A.',
'CORE_CPI' : 'Consumer price index (all items, less fresh food), S.A.'
}

## Make experiment with Foreign GDP +1 percent point. 

In [ ]:
instruments = [ 'V_NUSGAP','V_USGAP']
target      = baseline.loc['2005q1':,['USGDP','NUSGDP']].mfcalc('''\
USGDP  = USGDP*1.01
NUSGDP = NUSGDP*1.01
''',silent=1)
mqjem.invert(baseline,target,instruments); 

## Display the results 

In [ ]:
disp = mqjem['GDP CP INV EX IM CORE_CPI'].difpctlevel.mul100.rename(legend).plot(
    colrow=2,sharey=0,title='Impact of Foreign GDP +1 percent',top=0.9)

## Lower Oil prices  

In [ ]:
instruments = [ 'V_POIL']
target      = baseline.loc['2005q1':,['POIL']].mfcalc('''\
POIL  = POIL*0.9
''',silent=1)

resalt = mqjem.invert(baseline,target,instruments,silent=1)

In [ ]:
disp = mqjem['GDP CP INV EX IM CORE_CPI'].difpctlevel.rename(legend).plot(
    colrow=2,sharey=0,title='Impact of 10 percent permanent decrease in oil price',top=0.9)

## Combine the two previous experiments 

In [ ]:
instruments = [ 'V_POIL','V_NUSGAP','V_USGAP']
target      = baseline.loc['2005q1':,['POIL','USGDP','NUSGDP']].mfcalc('''\
POIL  = POIL*0.9
USGDP  = USGDP*1.01
NUSGDP = NUSGDP*1.01
''',silent=1)

resalt = mqjem.invert(baseline,target,instruments,silent=1)

In [ ]:
disp = mqjem['GDP CP INV EX IM CORE_CPI'].difpctlevel.mul100.rename(legend).plot(
    colrow=2,sharey=0,title='Impact of Foreign gdp GDP +1 and percent 10 percent permanent decrease in oil price',top=0.9)

## A permanent depreciation of exchange rates. 

In [ ]:
instruments = [ 'V_FXYEN']
target      = baseline.loc['2005q1':,['FXYEN']].mfcalc('''\
FXYEN = FXYEN*1.1
''',silent=1)

resalt = mqjem.invert(baseline,target,instruments,silent=1)

disp = mqjem['GDP CP INV EX IM CORE_CPI'].difpctlevel.mul100.rename(legend).plot(
    colrow=2,sharey=0,title='Impact of Foreign gdp GDP +1 and percent 10 percent permanent decrease in oil price',top=0.9)

## Draw the causal structure current periode, used when solving

In [ ]:
mqjem.plotadjacency(size=(19,19));  # delete the # in front to display, takes some time

## Draw the causal structuretaking all lags, the economic feedback  

In [ ]:
mqjem.plotadjacency(nolag=True,size=(19,19)); # delete the # in front to display, takes some time

## How is CPQ determined 

In [ ]:
mqjem.cpq.draw(up=2,down=2,HR=0,svg=1,transdic= {'ZPI*' : 'ZPI'}) # we condens all ZPI to one, to make the chart easy

## Also with values 
The result can be inspected in the graph/subfolder in PDF format. 

In [ ]:
with mqjem.set_smpl('2001q1','2001q3'):
    mqjem.cpq.draw(up=1,down=1,HR=0,svg=1,transdic= {'ZPI*' : 'ZPI'},last=1) # we condens all ZPI to one, to make the chart easy

## Another Example 
That determins the export (**EX**) and what is EX going into 

In [ ]:
mqjem.ex.draw(up=1,down=1)

# The values for EX
We only look at 3 quarters to make it simple

In [ ]:
with mqjem.set_smpl('2001q1','2001q3'):
    mqjem.ex.show